In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.special import erf

In [ ]:
def Area(xl, xh, yl, yh, sigmax, sigmay, Imax):
    # Calculates how much of a 2D Gaussian falls within a rectangular box
    ssigx = np.sqrt(2) * sigmax
    ssigy = np.sqrt(2) * sigmay    
    I = (erf(xh/ssigx)-erf(xl/ssigx))*(erf(yh/ssigy)-erf(yl/ssigy))
    return Imax * I / 4.0

class Array2d:
    def __init__(self,xmin,xmax,nx,ymin,ymax,ny):
        # This packages up an image which is nx * ny pixels
        self.nx=nx
        self.ny=ny

        self.xmin=xmin
        self.ymin=ymin
        
        self.xmax=xmax
        self.ymax=ymax
        
        self.dx=(xmax-xmin)/nx
        self.dy=(ymax-ymin)/ny
        
        self.x=np.linspace(xmin+self.dx/2,xmax-self.dx/2,nx)
        self.y=np.linspace(ymin+self.dy/2,ymax-self.dy/2,ny)

        self.data=np.zeros([nx,ny])


def BuildImage(nx, ny, sigmax, sigmay, Imax):
    spot = Array2d(-0.5, 0.5, nx,-0.5, 0.5, ny)
             
    for i in range(nx):
        for j in range(ny):
            xl = spot.x[i] - spot.dx / 2.0
            xh = xl + spot.dx
            yl = spot.y[j] - spot.dx / 2.0
            yh = yl + spot.dx
            spot.data[i,j] = Area(xl, xh, yl, yh, sigmax, sigmay, Imax)                
    return spot

In [ ]:
spot = BuildImage(50, 50, 0.10, 0.05, 100000)

In [ ]:
plt.imshow(spot.data, interpolation='nearest')

In [ ]:
def CalculateRotatedMoments(spot, theta):
    theta = theta * np.pi / 180.0
    sum = 0.0
    Ix = 0.0
    Iy = 0.0
    Ixx = 0.0
    Iyy = 0.0
    Ixy = 0.0
    for i in range(spot.nx):
        for j in range(spot.ny):
            x = spot.x[i]
            y = spot.y[j]
            xp = np.cos(theta) * x - np.sin(theta) * y
            yp = np.sin(theta) * x + np.cos(theta) * y
            sum += spot.data[i,j]
            Ix += xp * spot.data[i,j]
            Iy += yp * spot.data[i,j]
            Ixx += xp * xp * spot.data[i,j]
            Iyy += yp * yp * spot.data[i,j]
            Ixy += xp * yp * spot.data[i,j]
    Ix /= sum
    Iy /= sum
    Ixx /= sum
    Iyy /= sum
    Ixy /= sum
    
    print(f"Sum={sum:.1f}, Ix={Ix:.6f}, Iy={Iy:.6f}, Ixx={Ixx:.6f}, Iyy={Iyy:.6f}, Ixy={Ixy:.6f}")
    return [sum, Ix, Iy, Ixx, Iyy, Ixy]
            

def RotatedMoments(Ixx, Iyy, Ixy, theta):
    # Rotates the moments about an angle theta.
    # Formulae are fron the Sextractor documentation
    # https://sextractor.readthedocs.io/en/latest/Position.html\
    # ?highlight=shape#basic-shape-parameters-a-b-theta
    theta = theta * np.pi / 180.0
    c = np.cos(theta)
    s = np.sin(theta)
    IxxRot = c * c * Ixx + s * s * Iyy - 2.0 * c * s * Ixy
    IyyRot = s * s * Ixx + c * c * Iyy + 2.0 * c * s * Ixy
    IxyRot = c * s * (Ixx - Iyy) + (c * c - s * s) * Ixy
    return [IxxRot, IyyRot, IxyRot]  

In [ ]:
[sum, Ix, Iy, Ixx, Iyy, Ixy] = CalculateRotatedMoments(spot, 0.0)

In [ ]:
Ip = (Ixx + Iyy) / 2.0
Im = (Ixx - Iyy) / 2.0
A2 = Ip + np.sqrt(Im**2 + Ixy**2)
B2 = Ip - np.sqrt(Im**2 + Ixy**2)
phi = np.arctan2(Ixy , Im) / 2.0

print(A2, B2, phi * 180.0 / np.pi)

In [ ]:
theta = 22.0
[sum, Ixp, Iyp, Ixxp, Iyyp, Ixyp] = CalculateRotatedMoments(spot, theta)

In [ ]:
Ipp = (Ixxp + Iyyp) / 2.0
Imp = (Ixxp - Iyyp) / 2.0
A2p = Ipp + np.sqrt(Imp**2 + Ixyp**2)
B2p = Ipp - np.sqrt(Imp**2 + Ixyp**2)
phip = np.arctan2(Ixyp , Imp) / 2.0

print(A2p, B2p, phip * 180.0 / np.pi)

In [ ]:
# Going forward
[Ixxpp, Iyypp, Ixypp] = RotatedMoments(Ixx, Iyy, Ixy, theta)
print(Ixxpp, Iyypp, Ixypp)

In [ ]:
# Going back
[Ixx0, Iyy0, Ixy0] = RotatedMoments(Ixxp, Iyyp, Ixyp, -theta)
print(Ixx0, Iyy0, Ixy0)

In [ ]:
# All the stuff below is stuff I no longer need.

In [ ]:
theta

In [ ]:
# Now this should recover the moments about the original unrotated axis
IxxTest = ((A2p + B2p) + (A2p - B2p) / np.sqrt(1.0 + (np.tan(2.0 * phi))**2)) / 2.0
IyyTest = ((A2p + B2p) - (A2p - B2p) / np.sqrt(1.0 + (np.tan(2.0 * phi))**2)) / 2.0
print(IxxTest, IyyTest)

In [ ]:
# Now this should recover the moments about the original unrotated axis
SqTerm = np.sqrt(Imp**2 + Ixyp**2)
IxxTest2 = (Ipp + SqTerm / np.sqrt(1.0 + (np.tan(2.0 * phi))**2))
IyyTest2 = (Ipp - SqTerm / np.sqrt(1.0 + (np.tan(2.0 * phi))**2))
print(IxxTest2, IyyTest2)

In [ ]:
# Now this should calculate the moments about a rotated axis
thetaR = theta * np.pi / 180.0
SqTerm = np.sqrt(Im**2 + Ixy**2)
IxxTest3 = (Ip + SqTerm / np.sqrt(1.0 + (np.tan(2.0 * theta))**2))
IyyTest3 = (Ip - SqTerm / np.sqrt(1.0 + (np.tan(2.0 * theta))**2))
print(IxxTest3, IyyTest3)

In [ ]:
# This is just a test to see if the rotational moments are correct
thetaR = theta * np.pi / 180.0
IxxpTest = np.cos(thetaR)**2 * Ixx + np.sin(thetaR)**2 * Iyy - 2.0 * np.cos(thetaR) * np.sin(thetaR) * Ixy
IyypTest = np.sin(thetaR)**2 * Ixx + np.cos(thetaR)**2 * Iyy + 2.0 * np.cos(thetaR) * np.sin(thetaR) * Ixy
IxypTest = np.cos(thetaR) * np.sin(thetaR) * (Ixx - Iyy) + (np.cos(thetaR)**2 - np.sin(thetaR)**2) * Ixy

print(IxxpTest, IyypTest, IxypTest)
